# VAE Improvements. 

## How to improve variational family?

Last time we spoke about ordinary VAE. We saw, that it is quite simple model and sort of naive in its assumption about variational family.

Specifically, it assumes that posterior distribution of latent variables is unimodal factorized Gaussian, and it looks for such parameters $\theta$ of decoder, which maximize a lowerbound on marginal likelihood with limitations above. This could and leads to worse latent representation and worse generator model.

To address this limitation, today we will go through some improvements, which allow us to make posterior more expressive

In [ ]:
## First, import some models and functions we will use
from models import Base, VAE, IWAE, VAE_with_flows, VAE_MCMC
from main import make_dataloaders, get_activations
from models.samplers import HMC

## Visualization
from plotting import plot_digit_samples, plot_posterior
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib widget

## Math processing
import numpy as np
import torch
import torch.nn
from torchvision.transforms import ToTensor
## Wrapper on top of PyTorch, which ease work on neural nets
import pytorch_lightning as pl
## Is there is a cuda device on the machine, lets use it!
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

First, load data.

As previously, we will be using FashionMNIST dataset.

In [2]:
train_loader, val_loader = make_dataloaders(dataset='fashionmnist',
                                            gpus=1,
                                            batch_size=32,
                                            val_batch_size=64)

In [3]:
act_func = get_activations()

In [4]:
## Function to recover given images
def recover_image(model, pics, num_samples=50):
    with torch.no_grad():
        pics_rec = torch.sigmoid(model.step([pics.to(device), None])[1]).cpu().view(
                (num_samples, -1, 784)).mean(0).view((64, 28, 28))
    return pics_rec

## Let us fix the random sample, which we will use for image generation:
random_vector = torch.randn((64, 20)).to(device)

## And let us write a function to generate images:
def generate_image(model, random_vector):
    with torch.no_grad():
        generated = torch.sigmoid(model(random_vector)).cpu().view((64, 28, 28))
    return generated

def get_posterior_samples(model, pics, num_samples=50, hidden_dim=20):
    with torch.no_grad():
        z_samples = model.step([pics.to(device), None])[2].reshape(num_samples, -1, hidden_dim).mean(0).cpu()
    return z_samples

## Sample a batch from validation dataset
pics = None
labels = None
for b in val_loader:
    pics = b[0]
    labels = b[1]
    break
    
whole_dataset = val_loader.dataset.data[:1000] * 1. / torch.max(val_loader.dataset.data)
whole_labels = val_loader.dataset.targets[:1000]

Then, let us load already trained models (despite we can train it within the notebook, it takes some time). So just load them

In [5]:
vae = VAE.load_from_checkpoint(checkpoint_path='./checkpoints/VAE_None.ckpt', act_func=act_func['tanh'], num_samples=50, hidden_dim=20).to(device)
iwae = VAE.load_from_checkpoint(checkpoint_path='./checkpoints/IWAE_None.ckpt', act_func=act_func['tanh'], num_samples=50, hidden_dim=20).to(device)
vae_iaf = VAE_with_flows.load_from_checkpoint(checkpoint_path='./checkpoints/VAE_with_flows_IAF.ckpt', act_func=act_func['tanh'], num_samples=10, hidden_dim=20, flow_type='IAF', num_flows=5, need_permute=False).to(device)
vae_bnaf = VAE_with_flows.load_from_checkpoint(checkpoint_path='./checkpoints/VAE_with_flows_BNAF.ckpt', act_func=act_func['tanh'], num_samples=10, hidden_dim=20, flow_type='BNAF', num_flows=5, need_permute=False).to(device)
vae_rnvp = VAE_with_flows.load_from_checkpoint(checkpoint_path='./checkpoints/VAE_with_flows_RNVP.ckpt', act_func=act_func['tanh'], num_samples=10, hidden_dim=20, flow_type='RNVP', num_flows=5, need_permute=True).to(device)
vae_mcmc = VAE_MCMC.load_from_checkpoint(checkpoint_path='./checkpoints/VAE_MCMC_None.ckpt', act_func=act_func['tanh'], num_samples=10, hidden_dim=20, n_leapfrogs=5, step_size=0.1, use_barker=True).to(device)

# Vanilla VAE model

Let us recap the vanilla VAE model which we studies last time.

We were otimizing the following objective:

$$
\mathcal{L}_{\phi, \theta}(X) = \frac{1}{N}  \sum_{i=1}^{N} \mathcal{L}_{\phi, \theta}(x_i) = \frac{1}{N} \sum_{i=1}^{N} \int_{z} q_\phi (z|x_i) \log \frac{p_\theta(z, x_i)}{q_\phi(z|x_i)} dz =  \frac{1}{N} \sum_{i=1}^{N} ( \frac{1}{K} \sum_{k=1}^{K} p_\theta(x_i | z_k) - \text{KL}(q_\phi(z|x_i)\|p(z)) )
$$

In case of Gaussian variational family and prior, the last term could be computed in closed form, so reducing variance of the estimator.

The gradients could be computed using "reparametrization trick".

This is the code for model training. It requires only 3 lines of code.
```python
model = VAE(act_func=act_func['tanh'], num_samples=50, hidden_dim=20)
trainer = pl.Trainer(gpus=1)
trainer.fit(model, train_dataloader=train_loader, val_dataloaders=val_loader)
```

In [6]:
Base??

Init signature: Base(act_func, num_samples, hidden_dim, name='VAE')
Docstring:     
Helper class that provides a standard way to create an ABC using
inheritance.
Source:        
class Base(pl.LightningModule):
    def __init__(self, act_func, num_samples, hidden_dim, name="VAE"):
        super(Base, self).__init__()
        # Encoder
        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, hidden_dim)
        self.fc22 = nn.Linear(400, hidden_dim)
        # Decoder
        self.fc3 = nn.Linear(hidden_dim, 400)
        self.fc4 = nn.Linear(400, 784)

        self.act_func = act_func
        self.num_samples = num_samples
        # Fixed random vector, which we recover each epoch
        self.random_z = torch.randn((64, hidden_dim), dtype=torch.float32)

        self.name = name

    def encode(self, x):
        h1 = self.act_func(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
    

In [7]:
VAE??

Init signature: VAE(act_func, num_samples, hidden_dim, name='VAE')
Docstring:     
Helper class that provides a standard way to create an ABC using
inheritance.
Source:        
class VAE(Base):
    def loss_function(self, recon_x, x, mu, logvar):
        BCE = F.binary_cross_entropy_with_logits(recon_x, x.view(-1, 784), reduction='none').view(
            (self.num_samples, -1, 784)).mean(0).sum(-1).mean()
        KLD = -0.5 * torch.mean((1 + logvar - mu.pow(2) - logvar.exp()).view(
            (self.num_samples, -1, self.fc22.out_features)).mean(0).sum(-1))
        loss = BCE + KLD
        return loss

    def step(self, batch):
        x, _ = batch
        x = x.repeat(self.num_samples, 1, 1, 1)
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        x_hat = self(z)
        loss = self.loss_function(x_hat, x, mu, logvar)
        return loss, x_hat, z
File:           ~/github/vae_demo/models/VAE.py
Type:           ABCMeta
Subclasses:     


In [8]:
reconstructed = recover_image(vae, pics)
generated = generate_image(vae, random_vector)
plot_digit_samples(original=pics.squeeze(), reconstucted=reconstructed, generated=generated)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
z_posterior = get_posterior_samples(vae, whole_dataset.to(device))
plot_posterior([whole_labels.numpy(), z_posterior.cpu().numpy()])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# IWAE model

```python
model = IWAE(act_func=act_func['tanh'], num_samples=50)
trainer = pl.Trainer(gpus=1, deterministic=True)
trainer.fit(model, train_dataloader=train_loader, val_dataloaders=val_loader)
```

Let us now move to more expressive models. The first one -- Importance Weightd Autoencoders, or IWAE


The idea is the following: instead of maximizing the previous ELBO, we will optimize another one, which corresponds to the k-sample importance weighting
estimate of the log-likelihood:

$$
\mathcal{L}_{\phi, \theta}^{\text{IWAE}}(X) = \frac{1}{N}  \sum_{i=1}^{N} \mathcal{L}_{\phi, \theta}^{\text{IWAE}}(x_i) = \frac{1}{N} \sum_{i=1}^{N} \mathbb{E}_{q_\phi(z|x_i)} \left( \frac{1}{K} \sum_{k=1}^{K} \log \frac{p_\theta(z_k, x_i)}{q_\phi(z_k|x_i)} \right) = \frac{1}{N} \sum_{i=1}^{N} \int_{z} q_\phi (z|x_i) \frac{1}{K} \sum_{k=1}^{K} \log \frac{p_\theta(z_k, x_i)}{q_\phi(z_k|x_i)} dz
$$

In [10]:
IWAE??

Init signature: IWAE(act_func, num_samples, hidden_dim, name='VAE')
Docstring:     
Helper class that provides a standard way to create an ABC using
inheritance.
Source:        
class IWAE(Base):
    def loss_function(self, recon_x, x, mu, logvar, z):
        log_Q = torch.distributions.Normal(loc=mu,
                                           scale=torch.exp(0.5 * logvar)).log_prob(z).view(
            (self.num_samples, -1, self.fc22.out_features)).sum(-1)

        log_Pr = torch.sum((-0.5 * z ** 2).view((self.num_samples, -1, self.fc22.out_features)), -1)
        BCE = F.binary_cross_entropy_with_logits(recon_x, x.view(-1, 784), reduction='none').view(
            (self.num_samples, -1, 784)).sum(-1)
        log_weight = log_Pr + BCE - log_Q
        log_weight = log_weight - torch.max(log_weight, 0)[0]  # for stability
        weight = torch.exp(log_weight)
        weight = weight / torch.sum(weight, 0)
        weight = weight.detach()
        loss = torch.mean(torch.sum(weight * (-

In [11]:
reconstructed = recover_image(iwae, pics)
generated = generate_image(iwae, random_vector)
plot_digit_samples(original=pics.squeeze(), reconstucted=reconstructed, generated=generated)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
z_posterior = get_posterior_samples(iwae, whole_dataset.to(device))
plot_posterior([whole_labels.numpy(), z_posterior.cpu().numpy()])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# VAE model with flows

The next idea how we can improve on variational family is the usage of Normalizing flows.

It works as follows: given some input vector, it transform it in such a way, that the determinant of Jacobian this transformation is easy to compute.

TODO: Write formular, probably an example on NF on simple densities.

In [13]:
VAE_with_flows??

Init signature:
VAE_with_flows(
    act_func,
    num_samples,
    hidden_dim,
    flow_type,
    num_flows,
    need_permute,
)
Docstring:     
Helper class that provides a standard way to create an ABC using
inheritance.
Source:        
class VAE_with_flows(Base):
    def __init__(self, act_func, num_samples, hidden_dim, flow_type, num_flows, need_permute):
        super(VAE_with_flows, self).__init__(act_func, num_samples, hidden_dim)
        self.Flow = NormFlow(flow_type, num_flows, hidden_dim, need_permute)
        self.name += f'_{flow_type}_{num_flows}'

    def loss_function(self, recon_x, x, mu, logvar, z, z_transformed, log_jac):
        BCE = F.binary_cross_entropy_with_logits(recon_x, x.view(-1, 784), reduction='none').view(
            (self.num_samples, -1, 784)).sum(-1).mean()
        log_Q = torch.distributions.Normal(loc=mu, scale=torch.exp(0.5 * logvar)).log_prob(z)
        log_Pr = (-0.5 * z_transformed ** 2)
        KLD = torch.mean((log_Q - log_Pr).view(
         

## IAF

```python
model = VAE_with_flows(act_func=act_func['tanh'], num_samples=10, num_flows=5, flow_type="IAF", need_permute=False)
trainer = pl.Trainer(gpus=1, deterministic=True)
trainer.fit(model, train_dataloader=train_loader, val_dataloaders=val_loader)
```

In [14]:
reconstructed = recover_image(vae_iaf, pics, num_samples=10)
generated = generate_image(vae_iaf, random_vector)
plot_digit_samples(original=pics.squeeze(), reconstucted=reconstructed, generated=generated)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
z_posterior = get_posterior_samples(vae_iaf, whole_dataset.to(device), num_samples=10)
plot_posterior([whole_labels.numpy(), z_posterior.cpu().numpy()])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## BNAF

```python
model = VAE_with_flows(act_func=act_func['tanh'], num_samples=10, num_flows=5, flow_type="IAF", need_permute=False)
trainer = pl.Trainer(gpus=1, deterministic=True)
trainer.fit(model, train_dataloader=train_loader, val_dataloaders=val_loader)
```

In [16]:
reconstructed = recover_image(vae_bnaf, pics, num_samples=10)
generated = generate_image(vae_bnaf, random_vector)
plot_digit_samples(original=pics.squeeze(), reconstucted=reconstructed, generated=generated)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
z_posterior = get_posterior_samples(vae_bnaf, whole_dataset.to(device), num_samples=10)
plot_posterior([whole_labels.numpy(), z_posterior.cpu().numpy()])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## RNVP

```python
model = VAE_with_flows(act_func=act_func['tanh'], num_samples=10, num_flows=5, flow_type="RNVP", need_permute=True)
trainer = pl.Trainer(gpus=1, deterministic=True)
trainer.fit(model, train_dataloader=train_loader, val_dataloaders=val_loader)
```

In [18]:
reconstructed = recover_image(vae_rnvp, pics, num_samples=10)
generated = generate_image(vae_rnvp, random_vector)
plot_digit_samples(original=pics.squeeze(), reconstucted=reconstructed, generated=generated)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
z_posterior = get_posterior_samples(vae_rnvp, whole_dataset.to(device), num_samples=10)
plot_posterior([whole_labels.numpy(), z_posterior.cpu().numpy()])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# VAE with MCMC

The last idea -- add MCMC method to improve variational approximation.

The approach we consider utilizes an idea of decoupling training procedures for encoder and for decoder.



In [20]:
reconstructed = recover_image(vae_mcmc, pics, num_samples=10)
generated = generate_image(vae_mcmc, random_vector)
plot_digit_samples(original=pics.squeeze(), reconstucted=reconstructed, generated=generated)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
z_posterior = get_posterior_samples(vae_mcmc, whole_dataset.to(device), num_samples=10)
plot_posterior([whole_labels.numpy(), z_posterior.cpu().numpy()])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …